<a href="https://colab.research.google.com/github/AnjuHaliya/NNDL/blob/main/Assignment_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import Necessary Modules
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [3]:
# Read the dataset
data = pd.read_excel("/content/Load Data in kW.xlsx")
print(data.shape)
data.head()

(2184, 3)


,Date,Time,Load (kW)
0,01.09.2018,00:00:00,5551.82208
1,NaN,01:00:00,4983.17184
2,NaN,02:00:00,4888.39680
3,NaN,03:00:00,5072.95872
4,NaN,04:00:00,5196.25980


In [4]:
#Reshape the dataset
x = []
y = []
for i in range(24 * 7, data.shape[0]):
    x.append(data['Load (kW)'].iloc[i - 24 * 7]) #Since x is 24 * 7 hours before y
    y.append(data['Load (kW)'].iloc[i])

In [5]:
data = pd.DataFrame({'Previous Week Load' : x, 'Present Week Load' : y})
data.head()

,Previous Week Load,Present Week Load
0,5551.82208,5346.52812
1,4983.17184,5138.89596
2,4888.39680,4940.77248
3,5072.95872,4766.18688
4,5196.25980,5020.11540


In [6]:
# Data Normalization
maxX = np.max(data['Previous Week Load'])
minX = np.min(data['Previous Week Load'])
maxY = np.max(data['Present Week Load'])
minY = np.min(data['Present Week Load'])

data['Previous Week Load'] = (data['Previous Week Load'] - minX) / (maxX - minX)
data['Present Week Load'] = (data['Present Week Load'] - minY) / (maxY - minY)


In [7]:
# Splitting Data
x = data['Previous Week Load']
y = data['Present Week Load']
trainX, testX, trainY, testY = train_test_split(x, y, test_size=0.1, random_state=101)

In [8]:
# Model Training

#Initialization
m = np.random.uniform(-2, 2)
c = np.random.uniform(-2, 2)
eta = 0.5
epochs = 500
gm2 = 0
gc2 = 0
e = 1E-6

# Loop for epochs
for _ in range(epochs):
    
    for i in range(trainX.shape[0]):
        # Calculating Grad M
        gradM = -1 * (trainY.iloc[i] - m * trainX.iloc[i] - c) * trainX.iloc[i]
    
        # Calculating Grad C
        gradC = -1 * (trainY.iloc[i] - m * trainX.iloc[i] - c)
        
        # Calculating updated values of gm2 and gc2
        gm2 += (gradM ** 2)
        gc2 += (gradC ** 2)
        
        # Updating m and c values
        m -= (eta * gradM) / ((e + gm2) ** 0.5)
        c -= (eta * gradC) / ((e + gc2) ** 0.5)

# Print model parameters after training
print(f'm = {m}\nc = {c}')

m = 0.4857347967797122
c = 0.23135522564748853


In [10]:
# Calculating Predictions
train_pred = [m * trainX.iloc[i] + c for i in range(trainX.shape[0])]
test_pred = [m * testX.iloc[i] + c for i in range(testX.shape[0])]

# Denormalization of Output Values
train_pred = [i * (maxY - minY) + minY for i in train_pred]
test_pred = [i * (maxY - minY) + minY for i in test_pred]
trainY = [i * (maxY - minY) + minY for i in trainY]
testY = [i * (maxY - minY) + minY for i in testY]

# Training Error Calculation
training_MAE = sum([abs(train_pred[i] - trainY[i]) for i in range(len(trainY))]) / len(trainY)
training_MSE = sum([(train_pred[i] - trainY[i]) ** 2 for i in range(len(trainY))]) / len(trainY)
training_RMSE = training_MSE ** 0.5
print('Training Error :')
print(f'MAE : {training_MAE}')
print(f'MSE : {training_MSE}')
print(f'RMSE : {training_RMSE}')

# Testing Error Calculation
testing_MAE = sum([abs(test_pred[i] - testY[i]) for i in range(len(testY))]) / len(testY)
testing_MSE = sum([(test_pred[i] - testY[i]) ** 2 for i in range(len(testY))]) / len(testY)
testing_RMSE = testing_MSE ** 0.5
print('Testing Error :')
print(f'MAE : {testing_MAE}')
print(f'MSE : {testing_MSE}')
print(f'RMSE : {testing_RMSE}')

Training Error :
MAE : 737.0271910851939
MSE : 891424.9548666936
RMSE : 944.1530357239199
Testing Error :
MAE : 787.1173881714714
MSE : 968116.8755066507
RMSE : 983.9293041202964


In [11]:
# Checking predicted training values
pd.DataFrame({'Actual' : trainY, 'Predicted' : train_pred})

,Actual,Predicted
0,5086.98792,5060.570250
1,6378.60960,5731.516140
2,7647.78456,6182.146916
3,6165.05400,5656.057456
4,5918.45184,5501.806572
...,...,...
1809,5020.11540,5866.675068
1810,4048.82712,5524.156282
1811,5674.03200,5578.174397
1812,5621.65632,6995.524860


In [12]:
# Checking predicted testing values
pd.DataFrame({'Actual' : testY, 'Predicted' : test_pred})

,Actual,Predicted
0,7609.43808,6756.042029
1,4187.24856,5748.789815
2,6738.69240,6181.540822
3,5640.67368,6206.087623
4,6550.07760,5442.409375
...,...,...
197,5761.79244,6598.836437
198,6955.67736,6087.141706
199,8607.69360,6021.077476
200,6532.93080,6219.497450


In [13]:
# Real Time Prediction
x = float(input('Enter the load at same hour in previous week: '))
x = (x - minX) / (maxX - minX)
prediction = m * x + c
prediction = (prediction * (maxY - minY)) + minY
print('Predicted load at present hour :', prediction)

Enter the load at same hour in previous week: 6538.5732
Predicted load at present hour : 6176.555478889414
